In [141]:
from datetime import datetime

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import xgboost as xgb
import yfinance as yf

In [142]:
from warnings import simplefilter
simplefilter(action="ignore", category=pd.errors.PerformanceWarning)

In [143]:
df = pd.read_parquet("../data/stock_data.parquet")
events = pd.read_parquet("../data/events.parquet")
dividends = pd.read_parquet("../data/dividends.parquet")
splits = pd.read_parquet("../data/splits.parquet")
# df

In [144]:

def fib_retracement(data: pd.DataFrame, high: str, low: str, target_column: str, levels: list = [0.236, 0.382, 0.618, 1.0]):
    for level in levels:
        data[f"{target_column}_fib_{level}"] = data[high] - (data[high] - data[low]) * level

    return data


def create_lag_features(data: pd.DataFrame, target_column, lag_steps=1):
    if isinstance(lag_steps, int):
        for i in range(1, lag_steps + 1):
            data[f"{target_column}_lag_{i}"] = data[target_column].shift(i)

    if isinstance(lag_steps, list):
        for i in lag_steps:
            data[f"{target_column}_lag_{i}"] = data[target_column].shift(i)

    return data


def simple_moving_average(data: pd.DataFrame, target_column: str | list, window_size: int | list = 3):
    if isinstance(target_column, list) and isinstance(window_size, list):
        for col in target_column:
            for window in window_size:
                data[f"{col}_rolling_mean_{window}"] = data[col].rolling(window=window).mean()

    elif isinstance(target_column, str):
        data[f"{target_column}_rolling_mean_{window_size}"] = data[target_column].rolling(window=window_size).mean()

    elif isinstance(target_column, list):
        for col in target_column:
            data[f"{col}_rolling_mean_{window_size}"] = data[col].rolling(window=window_size).mean()

    return data


def exponential_moving_average(data: pd.DataFrame, target_column: str | list, window_size: int | list = 3):
    if isinstance(target_column, str):
        target_column = [target_column]
    if isinstance(window_size, int):
        window_size = [window_size]

    for col in target_column:
        for window in window_size:
            data[f"{col}_rolling_exp_mean_{window}"] = data[col].ewm(span=window).mean()

    return data


def bollinger_bands(data: pd.DataFrame, target_column: str, window_size: int = 20):
    data[f"{target_column}_rolling_mean_{window_size}"] = data[target_column].rolling(window=window_size).mean()
    data[f"{target_column}_rolling_std_{window_size}"] = data[target_column].rolling(window=window_size).std()
    data[f"{target_column}_bollinger_upper_{window_size}"] = data[f"{target_column}_rolling_mean_{window_size}"] + 2 * data[
        f"{target_column}_rolling_std_{window_size}"]
    data[f"{target_column}_bollinger_lower_{window_size}"] = data[f"{target_column}_rolling_mean_{window_size}"] - 2 * data[
        f"{target_column}_rolling_std_{window_size}"]

    return data

# def plot_bollinger_bands(data: pd.DataFrame, target_column: str, window_size: int = 20):
#     data[f"{target_column}_rolling_mean_{window_size}"] = data[target_column].rolling(window=window_size).mean()
#     data[f"{target_column}_rolling_std_{window_size}"] = data[target_column].rolling(window=window_size).std()
#     data[f"{target_column}_bollinger_upper_{window_size}"] = data[f"{target_column}_rolling_mean_{window_size}"] + 2 * data[
#         f"{target_column}_rolling_std_{window_size}"]
#     data[f"{target_column}_bollinger_lower_{window_size}"] = data[f"{target_column}_rolling_mean_{window_size}"] - 2 * data[
#         f"{target_column}_rolling_std_{window_size}"]

#     plt.figure(figsize=(12, 6))
#     plt.plot(data[target_column], label="Close Price", color="blue")
#     plt.plot(data[f"{target_column}_rolling_mean_{window_size}"], label="Rolling Mean", color="red")
#     plt.plot(data[f"{target_column}_bollinger_upper_{window_size}"], label="Bollinger Upper", color="green")
#     plt.plot(data[f"{target_column}_bollinger_lower_{window_size}"], label="Bollinger Lower", color="green")
#     plt.title(f"{target_column} Bollinger Bands")
#     plt.legend()
#     plt.show()

#     return data


def average_true_range(data: pd.DataFrame, window_size: int = 14):
    data[f"high_low_{window_size}"] = data["high"] - data["low"]
    data[f"high_close_{window_size}"] = np.abs(data["high"] - data["close"].shift(1))
    data[f"low_close_{window_size}"] = np.abs(data["low"] - data["close"].shift(1))
    data[f"true_range_{window_size}"] = np.max(
        [data[f"high_low_{window_size}"], data[f"high_close_{window_size}"], data[f"low_close_{window_size}"]], axis=0)
    data[f"average_true_range_{window_size}"] = data[f"true_range_{window_size}"].rolling(window=window_size).mean()

    return data


def relative_strength_index(data: pd.DataFrame, column: str = "close", window_size: int = 14):
    delta = data[column].diff()
    gain = (delta.where(delta > 0, 0)).rolling(window=window_size).mean()
    loss = (-delta.where(delta < 0, 0)).rolling(window=window_size).mean()
    rs = gain / loss
    data[f"rsi_{window_size}"] = 100 - (100 / (1 + rs))

    return data


def macd(data: pd.DataFrame, column: str = "close", short_window: int = 12, long_window: int = 26, signal_window: int = 9):
    data[f"short_ema_{short_window}"] = data[column].ewm(span=short_window, adjust=False).mean()
    data[f"long_ema_{long_window}"] = data[column].ewm(span=long_window, adjust=False).mean()
    data[f"macd_{short_window}_{long_window}"] = data[f"short_ema_{short_window}"] - data[f"long_ema_{long_window}"]
    data[f"signal_{signal_window}"] = data[f"macd_{short_window}_{
        long_window}"].ewm(span=signal_window, adjust=False).mean()
    data[f"macd_hist_{short_window}_{long_window}_{signal_window}"] = data[f"macd_{
        short_window}_{long_window}"] - data[f"signal_{signal_window}"]

    return data


def stochastic_oscillator(data: pd.DataFrame, window_size: int = 14):
    data[f"stochastic_oscillator_{window_size}"] = (data["close"] - data["low"].rolling(window=window_size).min()) / (
        data["high"].rolling(window=window_size).max() - data["low"].rolling(window=window_size).min())

    return data


def williams_r(data: pd.DataFrame, window_size: int = 14):
    data[f"williams_r_{window_size}"] = (data["high"].rolling(window=window_size).max() - data["close"]) / (
        data["high"].rolling(window=window_size).max() - data["low"].rolling(window=window_size).min())

    return data


def money_flow_index(data: pd.DataFrame, window_size: int = 14):
    typical_price = (data["high"] + data["low"] + data["close"]) / 3
    raw_money_flow = typical_price * data["volume"]
    money_flow_ratio = raw_money_flow.rolling(window=window_size).sum() / (typical_price * data["volume"]).rolling(
        window=window_size).sum()
    data[f"money_flow_index_{window_size}"] = 100 - (100 / (1 + money_flow_ratio))

    return data


def on_balance_volume(data: pd.DataFrame, offset: int = 1):
    data[f"on_balance_volume_{offset}"] = np.where(
        data["close"] > data["close"].shift(offset), data["volume"], -data["volume"])
    data[f"on_balance_volume_{offset}"] = data[f"on_balance_volume_{offset}"].cumsum()

    return data


def volume_weighted_average_price(data: pd.DataFrame, window_size: int = 14):
    data[f"volume_weighted_average_price_{window_size}"] = (
        data["close"] * data["volume"]).rolling(window=window_size).sum() / data["volume"].rolling(window=window_size).sum()

    return data


def volitility(data: pd.DataFrame, column: str = "close", window_size: int = 14):
    data[f"volitility_{column}_{window_size}"] = data[column].rolling(window=window_size).std()
    data[f"volitility_pct_change_{window_size}_{column}"] = data[column].pct_change().rolling(window=window_size).std()

    return data



def apply_technical_indicators(data: pd.DataFrame, target_column: str | list = "close", window_size: int | list = 14):

    if isinstance(target_column, str):
        target_column = [target_column]
    if isinstance(window_size, int):
        window_size = [window_size]

    for col in target_column:
        for window in window_size:
            data = volitility(data, col, window)
            data = average_true_range(data, window)
            data = simple_moving_average(data, col, window)
            data = exponential_moving_average(data, col, window)
            data = relative_strength_index(data, col, window)
            data = macd(data, col)
            data = stochastic_oscillator(data, window)
            data = williams_r(data, window)
            data = money_flow_index(data, window)
            data = on_balance_volume(data)
            data = volume_weighted_average_price(data, window)
            data = bollinger_bands(data, col, window)
            data = fib_retracement(data, "high", "low", col)

    return data


def forward_fill_with_decay(df: pd.DataFrame, column, decay_factor):
    """
    Forward fills missing values in a specified column with a decay factor applied.

    Parameters:
    - df: The DataFrame containing the data.
    - column: The column name (or index) to apply forward fill with decay.
    - decay_factor: The factor by which the previous value decays (0 < decay_factor < 1).

    Returns:
    - The DataFrame with missing values filled with decayed forward fill.
    """
    if not 0 < decay_factor < 1:
        raise ValueError("Decay factor must be between 0 and 1.")

    df = df.copy()  # Avoid modifying the original DataFrame
    df[column] = df[column].astype(float)  # Ensure column is of float type for NaNs

    # Initialize variables
    last_value = np.nan
    decay = 1.0

    for i in range(len(df)):
        if pd.notna(df.at[i, column]):
            # Update the last value and reset decay
            last_value = df.at[i, column]
            decay = 1.0
            df.at[i, column] = last_value * decay
            decay *= decay_factor  # Apply the decay factor for the next value

        elif pd.isna(df.at[i, column]) and pd.notna(last_value):
            # Apply decay to the last value and fill the current cell
            df.at[i, column] = last_value * decay
            decay *= decay_factor  # Apply the decay factor for the next value

    return df


def apply_event(data: pd.DataFrame, event: pd.DataFrame, prefix_name, falloff: bool = True, decay_factor: float = 0.99) -> pd.DataFrame:
    """Join Events df to data df based on Date"""
    event = event.copy(deep=True)

    event["date"] = pd.to_datetime(event["date"])

    # Rename Events columns to event_* to avoid conflicts
    for column in event.columns:
        if column != "date":
            event.rename(columns={column: f"event_{column}"}, inplace=True)

    df = data.merge(event, on="date", how="left").copy(deep=True)

    df["event_decay"] = np.where(pd.notna(df["event_name"]), 1, np.nan)
    df["event_name"] = df["event_name"].ffill()
    df["event_value"] = df["event_value"].ffill()
    df["event_sentiment"] = df["event_sentiment"].ffill()

    df = df.rename(columns={
        "event_name": f"{prefix_name}_event_name",
        "event_value": f"{prefix_name}_event_value",
        "event_sentiment": f"{prefix_name}_event_sentiment",
        "event_decay": f"{prefix_name}_event_decay"
    })

    # forward fill sentiment with decay
    if falloff:
        df = forward_fill_with_decay(df, f"{prefix_name}_event_decay", decay_factor)

    return df


def day_of_week(data: pd.DataFrame) -> pd.DataFrame:
    data["day_of_week"] = data["date"].dt.dayofweek.astype(int)
    return data


def day_of_month(data: pd.DataFrame) -> pd.DataFrame:
    data["day_of_month"] = data["date"].dt.day.astype(int)
    return data


def month(data: pd.DataFrame) -> pd.DataFrame:
    data["month"] = data["date"].dt.month.astype(int)
    return data


def year(data: pd.DataFrame) -> pd.DataFrame:
    data["year"] = data["date"].dt.year.astype(int)
    return data


def week_of_year(data: pd.DataFrame) -> pd.DataFrame:
    data["week_of_year"] = data["date"].dt.isocalendar().week.astype(int)
    return data


def is_not_nan_column_and_default(data: pd.DataFrame, column) -> pd.DataFrame:
    if isinstance(column, list):
        for col in column:
            data[f"is_nan_{col}"] = data[col].notna().astype(int)
            data[col] = data[col].fillna(0)
        return data

    if isinstance(column, str):
        data[f"is_nan_{column}"] = data[column].notna().astype(int)
        data[column] = data[column].fillna(0)
        return data


def future_value(data: pd.DataFrame, column: str, offset: int, drop_recent: bool = True) -> pd.DataFrame:
    data[f"target_{column}"] = data[column].shift(-offset)
    # Drop last X offset rows
    if drop_recent:  # Drop to remove NaNs
        data = drop_recent_with_offset(data, offset)

    return data


def drop_recent_with_offset(data: pd.DataFrame, offset: int) -> pd.DataFrame:
    return data[:-offset]


def percent_change_between_columns(data: pd.DataFrame, column: str, column2: str) -> pd.DataFrame:
    data[f"percent_change_{column}"] = (data[column2] - data[column]) / data[column]
    return data


def percent_change(data: pd.DataFrame, column: str, offset: int | list = 1, drop_recent: bool = True) -> pd.DataFrame:
    if isinstance(offset, int):
        offset = [offset]

    for off in offset:
        data[f"{column}_percent_change_{off}"] = (data[column].shift(-off) - data[column]) / data[column]

    if drop_recent:  # Drop to remove NaNs
        data = drop_recent_with_offset(data, max(offset))

    return data

def up_down(data: pd.DataFrame, column: str, offset: int, drop_recent: bool = True) -> pd.DataFrame:
    """
    Simply tells wheither the price went up or down in the last window_size days
        Returns 1 if the price went up, 0 if the price went down, and 0 if the price stayed the same
    """
    if isinstance(offset, int):
        offset = [offset]

    for off in offset:
        data[f"up_down_{column}_{off}"] = np.where(data[column].shift(-off) > data[column], 1,
                                                    np.where(data[column].shift(-off) < data[column], 0, 0))
        
    if drop_recent:  # Drop to remove NaNs
        data = drop_recent_with_offset(data, max(offset))

    return data



In [145]:
# df

In [146]:
df = up_down(df, "close", 1)
# df

In [147]:
df = apply_technical_indicators(df, ["open", "close", "high", "low", "volume"], [7, 14, 30])
df

,date,open,high,low,close,adj close,volume,up_down_close_1,volitility_open_7,volitility_pct_change_7_open,...,volume_rolling_std_14,volume_bollinger_upper_14,volume_bollinger_lower_14,volitility_volume_30,volitility_pct_change_30_volume,volume_rolling_mean_30,volume_rolling_exp_mean_30,volume_rolling_std_30,volume_bollinger_upper_30,volume_bollinger_lower_30
0,2004-09-07,0.107500,0.109750,0.103417,0.104833,0.096156,727332000,0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,7.273320e+08,NaN,NaN,NaN
1,2004-09-08,0.104917,0.105833,0.102417,0.102667,0.094169,834984000,1,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,7.829522e+08,NaN,NaN,NaN
2,2004-09-09,0.105167,0.114750,0.104667,0.112917,0.103571,1299324000,1,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,9.666743e+08,NaN,NaN,NaN
3,2004-09-10,0.111000,0.117833,0.110667,0.117250,0.107545,1039344000,1,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,9.866974e+08,NaN,NaN,NaN
4,2004-09-13,0.117083,0.121833,0.116917,0.119500,0.109609,967752000,1,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,9.823869e+08,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5027,2024-08-27,125.050003,129.199997,123.879997,128.300003,128.300003,303134600,0,2.234831,0.028831,...,3.354272e+07,3.879217e+08,2.537508e+08,9.136152e+07,0.339751,3.506721e+08,3.407312e+08,9.136152e+07,5.333952e+08,1.679491e+08
5028,2024-08-28,128.119995,128.330002,122.639999,125.610001,125.610001,448101100,0,1.828960,0.029369,...,4.432903e+07,4.135079e+08,2.361918e+08,9.282680e+07,0.318593,3.526060e+08,3.476583e+08,9.282680e+07,5.382596e+08,1.669524e+08
5029,2024-08-29,121.360001,124.430000,116.709999,117.589996,117.589996,453023300,1,2.988338,0.033212,...,5.472950e+07,4.458931e+08,2.269751e+08,9.439277e+07,0.315926,3.570074e+08,3.544560e+08,9.439277e+07,5.457930e+08,1.682219e+08
5030,2024-08-30,119.529999,121.750000,117.220001,119.370003,119.370003,333751600,0,4.017185,0.033336,...,5.464803e+07,4.463153e+08,2.277232e+08,9.077038e+07,0.313661,3.608917e+08,3.531203e+08,9.077038e+07,5.424324e+08,1.793509e+08


In [148]:
# %%capture
# df = percent_change(df, "close", [1, 7, 14, 30])
# df = future_value(df, "close", 1)
# df.tail(10)

In [149]:

df = apply_event(df, dividends, prefix_name="dividends", falloff=True, decay_factor=0.95)
df = apply_event(df, splits, prefix_name="splits", falloff=True, decay_factor=0.95)
df = is_not_nan_column_and_default(df, ["dividends_event_value", "dividends_event_sentiment"])
df = is_not_nan_column_and_default(df, ["splits_event_value", "splits_event_sentiment"])
# df

df = create_lag_features(df, "open", lag_steps=list(range(1, 30)))
df = create_lag_features(df, "close", lag_steps=list(range(1, 30)))
df = create_lag_features(df, "high", lag_steps=list(range(1, 30)))
df = create_lag_features(df, "low", lag_steps=list(range(1, 30)))
df = create_lag_features(df, "volume", lag_steps=list(range(1, 30)))
# df = create_lag_features(df, "close_percent_change_1", lag_steps=[1, 2, 3, 4, 5, 6, 7, 10, 14, 21, 30, 60])

df = day_of_week(df)
df = day_of_month(df)
df = month(df)
df = year(df)
df = week_of_year(df)


# df = create_lag_features(df, "event_name", lag_steps=list(range(1, 60)))
df

,date,open,high,low,close,adj close,volume,up_down_close_1,volitility_open_7,volitility_pct_change_7_open,...,volume_lag_25,volume_lag_26,volume_lag_27,volume_lag_28,volume_lag_29,day_of_week,day_of_month,month,year,week_of_year
0,2004-09-07,0.107500,0.109750,0.103417,0.104833,0.096156,727332000,0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,1,7,9,2004,37
1,2004-09-08,0.104917,0.105833,0.102417,0.102667,0.094169,834984000,1,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,2,8,9,2004,37
2,2004-09-09,0.105167,0.114750,0.104667,0.112917,0.103571,1299324000,1,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,3,9,9,2004,37
3,2004-09-10,0.111000,0.117833,0.110667,0.117250,0.107545,1039344000,1,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,4,10,9,2004,37
4,2004-09-13,0.117083,0.121833,0.116917,0.119500,0.109609,967752000,1,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,0,13,9,2004,38
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5027,2024-08-27,125.050003,129.199997,123.879997,128.300003,128.300003,303134600,0,2.234831,0.028831,...,173911000.0,258068900.0,217223800.0,320979500.0,390086200.0,1,27,8,2024,35
5028,2024-08-28,128.119995,128.330002,122.639999,125.610001,125.610001,448101100,0,1.828960,0.029369,...,327776900.0,173911000.0,258068900.0,217223800.0,320979500.0,2,28,8,2024,35
5029,2024-08-29,121.360001,124.430000,116.709999,117.589996,117.589996,453023300,1,2.988338,0.033212,...,460067000.0,327776900.0,173911000.0,258068900.0,217223800.0,3,29,8,2024,35
5030,2024-08-30,119.529999,121.750000,117.220001,119.370003,119.370003,333751600,0,4.017185,0.033336,...,293399100.0,460067000.0,327776900.0,173911000.0,258068900.0,4,30,8,2024,35


In [150]:
df.filter(like= "open")

,open,volitility_open_7,volitility_pct_change_7_open,open_rolling_mean_7,open_rolling_exp_mean_7,open_rolling_std_7,open_bollinger_upper_7,open_bollinger_lower_7,open_fib_0.236,open_fib_0.382,...,open_lag_20,open_lag_21,open_lag_22,open_lag_23,open_lag_24,open_lag_25,open_lag_26,open_lag_27,open_lag_28,open_lag_29
0,0.107500,NaN,NaN,NaN,0.107500,NaN,NaN,NaN,0.108255,0.107331,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0.104917,NaN,NaN,NaN,0.106024,NaN,NaN,NaN,0.105027,0.104528,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,0.105167,NaN,NaN,NaN,0.105653,NaN,NaN,NaN,0.112370,0.110898,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,0.111000,NaN,NaN,NaN,0.107609,NaN,NaN,NaN,0.116142,0.115096,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,0.117083,NaN,NaN,NaN,0.110714,NaN,NaN,NaN,0.120673,0.119955,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5027,125.050003,2.234831,0.028831,127.214287,125.508177,2.234831,131.683948,122.744626,127.944477,127.167757,...,111.519997,113.690002,116.190002,113.040001,119.169998,122.779999,120.349998,120.349998,121.849998,121.349998
5028,128.119995,1.828960,0.029369,127.762858,126.161132,1.828960,131.420777,124.104939,126.987161,126.156421,...,112.900002,111.519997,113.690002,116.190002,113.040001,119.169998,122.779999,120.349998,120.349998,121.849998
5029,121.360001,2.988338,0.033212,126.757144,124.960849,2.988338,132.733820,120.780469,122.608080,121.480960,...,117.529999,112.900002,111.519997,113.690002,116.190002,113.040001,119.169998,122.779999,120.349998,120.349998
5030,119.529999,4.017185,0.033336,125.644287,123.603136,4.017185,133.678657,117.609917,120.680920,120.019540,...,103.760002,117.529999,112.900002,111.519997,113.690002,116.190002,113.040001,119.169998,122.779999,120.349998


In [151]:
df.to_parquet("../data/dataset.parquet")